
# Documento: Uso de `torch.expand` en PyTorch

## Introducción a `torch.expand`

La función `torch.expand` en PyTorch permite "expandir" un tensor a una nueva forma, replicando sus datos a lo largo de las dimensiones especificadas **sin copiar la información en memoria**. Esta función es muy útil cuando necesitamos trabajar con datos en dimensiones superiores, sin duplicar realmente el contenido en la memoria, lo cual ahorra espacio y mejora la eficiencia.

Sin embargo, `torch.expand` solo puede expandir dimensiones que originalmente tienen tamaño `1` (dimensiones *singleton*). Si intentas expandir una dimensión que ya tiene un tamaño mayor a `1`, PyTorch lanzará un error. 

A continuación, vamos a analizar varios ejemplos que muestran cómo funciona `torch.expand`, los problemas comunes que pueden surgir, cómo resolverlos y las reglas adicionales sobre la expansión al añadir dimensiones nuevas al inicio.

---

## Ejemplo 1: Expansión sin éxito en un tensor unidimensional

```python
tensor = torch.rand(5)  # Genera un tensor de forma [5]
expanded_tensor = tensor.expand(5, 3)
```

### Explicación

1. **Forma del tensor**: `tensor` tiene la forma `[5]`, es decir, es un tensor unidimensional con 5 elementos.
2. **Intención de expansión**: Intentamos expandir `tensor` a una nueva forma `[5, 3]`, agregando una dimensión adicional con 3 columnas.
3. **Por qué falla**: La función `expand` solo puede aumentar dimensiones que tienen tamaño `1`. Aquí, `tensor` tiene tamaño `5` en la primera dimensión y no puede expandirse a `[5, 3]` porque no existe una dimensión singleton que permita esta expansión.

### Error obtenido

Este código produce un error:

```
RuntimeError: The expanded size of the tensor (3) must match the existing size (5) at non-singleton dimension 1.
```

### Solución

Para que esto funcione, necesitamos agregar una dimensión singleton en el tensor original. Esto se puede hacer usando `.unsqueeze(1)`, que añade una dimensión de tamaño `1` en el eje especificado:

```python
expanded_tensor = tensor.unsqueeze(1).expand(5, 3)
```

Al usar `unsqueeze(1)`, cambiamos la forma de `tensor` de `[5]` a `[5, 1]`, lo cual permite que `expand` replique la dimensión `1` a `3`, generando el tensor expandido con forma `[5, 3]`.

---

## Ejemplo 2: Expansión exitosa en un tensor unidimensional con tamaño `[3]`

```python
tensor2 = torch.tensor([1, 2, 3])  # Tensor de forma [3]
expanded_tensor = tensor2.expand(2, 3)
```

### Explicación

1. **Forma del tensor**: `tensor2` tiene la forma `[3]`, es un tensor unidimensional con 3 elementos.
2. **Intención de expansión**: Intentamos expandir `tensor2` a una forma `[2, 3]`.
3. **Por qué funciona**: Aunque `tensor2` parece tener la forma `[3]`, PyTorch internamente lo interpreta como si tuviera una forma `[1, 3]` al aplicar `expand`. Esto significa que PyTorch asume una dimensión singleton (dimensión de tamaño `1`) en el primer eje, y permite que `expand(2, 3)` convierta este `[1, 3]` en `[2, 3]`, replicando los datos a lo largo de la nueva dimensión de tamaño `2`.

### Resultado

El código genera el siguiente tensor:

```
tensor([[1, 2, 3],
        [1, 2, 3]])
```

PyTorch ha replicado los datos de `tensor2` a lo largo de la primera dimensión, generando una nueva forma `[2, 3]`.

---

## Ejemplo 3: Tensor con valores diferentes y expansión exitosa

```python
tensor3 = torch.tensor([2, 2, 3])  # Tensor de forma [3]
expanded_tensor = tensor3.expand(2, 3)
```

### Explicación

Este caso es similar al Ejemplo 2.

1. **Forma del tensor**: `tensor3` tiene la forma `[3]`.
2. **Intención de expansión**: Intentamos expandirlo a `[2, 3]`.
3. **Por qué funciona**: Como en el Ejemplo 2, PyTorch interpreta el tensor `[3]` como `[1, 3]` para efectos de expansión. Esto permite que el tamaño `1` implícito en el primer eje se expanda a `2`, generando un tensor de forma `[2, 3]`.

### Resultado

El código genera:

```
tensor([[2, 2, 3],
        [2, 2, 3]])
```

Los valores `[2, 2, 3]` se replican a lo largo de la primera dimensión, formando un tensor con la nueva forma `[2, 3]`.

---

## Caso especial: Añadir una nueva dimensión al inicio de un tensor

Si deseas expandir un tensor añadiendo una nueva dimensión al inicio, PyTorch permite hacerlo sin necesidad de añadir manualmente una dimensión singleton. Esta regla es útil y permite simplificar el código en ciertos casos.

### Ejemplo 4: Expansión exitosa al añadir una dimensión al inicio

```python
tensor = torch.rand(5) 
expanded_tensor = tensor.expand(3, 5)
```

1. **Forma del tensor**: `tensor` tiene la forma `[5]`.
2. **Intención de expansión**: Queremos expandir `tensor` a la forma `[3, 5]`.
3. **Por qué funciona**: PyTorch permite añadir una dimensión singleton implícita al inicio del tensor, tratándolo como si fuera `[1, 5]`. Luego, expande esta primera dimensión `1` a `3`, resultando en un tensor de forma `[3, 5]`.

### Resultado

El código produce el siguiente tensor:

```
tensor([[x1, x2, x3, x4, x5],
        [x1, x2, x3, x4, x5],
        [x1, x2, x3, x4, x5]])
```

Donde `x1, x2, ..., x5` representan los valores originales del tensor `[5]`. Esto replica los valores en la nueva dimensión sin necesidad de añadir explícitamente una dimensión singleton.

### Regla general para múltiples dimensiones

Esta interpretación de añadir dimensiones singleton implícitas se aplica también a tensores de mayor dimensión, siempre que la expansión implique la adición de una nueva dimensión al inicio.

#### Ejemplo 5: Expansión con múltiples dimensiones

Si tienes un tensor bidimensional de forma `[5, 4]` y quieres expandirlo a `[3, 5, 4]`, esto funcionará sin errores:

```python
tensor = torch.rand(5, 4)  # Forma inicial [5, 4]
expanded_tensor = tensor.expand(3, 5, 4)
```

1. **Forma inicial**: `tensor` tiene la forma `[5, 4]`.
2. **Intención de expansión**: Queremos expandirlo a `[3, 5, 4]`.
3. **Por qué funciona**: PyTorch interpreta que `tensor` puede tratarse como `[1, 5, 4]`, lo cual permite que el tamaño `1` en el primer eje se expanda a `3`, obteniendo la forma final `[3, 5, 4]`.

### Resultado

El resultado será un tensor donde cada "capa" de la primera dimensión es una copia del tensor original `[5, 4]`.

---

## Reglas generales para usar `torch.expand`

1. **Expansión de dimensiones singleton**: `torch.expand` solo puede replicar dimensiones que tienen tamaño `1`. Si intentas expandir una dimensión que tiene un tamaño mayor que `1`, PyTorch lanzará un error.
   
2. **Interpretación de tensores unidimensionales**: Cuando trabajas con un tensor unidimensional como `[n]`, PyTorch lo interpreta como `[1, n]` al aplicar `expand`. Esto permite agregar una nueva dimensión al principio y realizar la expansión deseada en algunos casos.

3. **Añadir dimensiones al inicio**: Si deseas expandir un tensor añadiendo nuevas dimensiones al inicio (antes de las dimensiones existentes), PyTorch puede asumir automáticamente dimensiones singleton en esas posiciones y permitir la expansión.

4. **Expansión solo de dimensiones singleton**: Si intentas expandir una dimensión que ya tiene un tamaño mayor a `1` y no es la primera dimensión añadida, se producirá un error.

---

### Ejemplo final con solución alternativa

Si tienes un tensor como `[5]` y necesitas expandirlo a `[5, 3]`, puedes resolverlo de dos formas sin modificar el tensor original:

1. **Usando `reshape` para agregar una dimensión singleton temporalmente**:
   ```python
   expanded_tensor = tensor.reshape(5, 1).expand(5, 3

)
   ```

2. **Usando `unsqueeze` temporalmente**:
   ```python
   expanded_tensor = tensor.unsqueeze(1).expand(5, 3)
   ```

Ambas opciones añaden una dimensión singleton temporal, permitiendo que `expand` funcione sin modificar el tensor original.
